In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
#from lightgbm import LGBMRegressor
import warnings
warnings.filterwarnings("ignore")

In [ ]:
def get_avg(_id_):
    return average.loc[_id_]

def add_rank(df, col_name="pred"):
    df["Rank"] = df.groupby("Date")[col_name].rank(ascending=False, method="first") - 1 
    df["Rank"] = df["Rank"].astype("int")
    return df

def data_sets(prices):
    X = prices[['Date','SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume','Target', 'Avg']].dropna()
    codes = X.SecuritiesCode.unique()
    X['Var'] = (X['Close'] - X['Open']) * X['Volume']
    X['H-L'] = (X['High'] - X['Low']) * X['Volume']
    y = X.Target
    X['Date'] = pd.to_datetime(X['Date'])
    X['Date'] = X['Date'].dt.strftime("%Y%m%d").astype(int)
    X = X[['Date', 'H-L', 'Var', 'Open', 'Volume', 'Avg']]
    return X, y, codes

def predict_sets(prices):
    X = prices[['Date','SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume', 'Avg']]
    missing_val_count_by_column = (X.isnull().sum())
    print(missing_val_count_by_column)
    
def predict(prices):
    X = prices[['Date','SecuritiesCode', 'Open', 'High', 'Low', 'Close', 'Volume', 'Avg']]
    X['Date'] = pd.to_datetime(X['Date'])
    X['Date'] = X['Date'].dt.strftime("%Y%m%d").astype(int)
    # Imputation
    my_imputer = SimpleImputer()
    imputed_X = pd.DataFrame(my_imputer.fit_transform(X))
    imputed_X.columns = X.columns
    X = imputed_X
    X['Var'] = (X['Close'] - X['Open']) * X['Volume']
    X['H-L'] = (X['High'] - X['Low'])* X['Volume']
    X = X[['Date', 'H-L', 'Var','Open', 'Volume', 'Avg']]
    return X

def run_model(X):
    return model.predict(X)

In [ ]:
prices = pd.read_csv("../input/jpx-tokyo-stock-exchange-prediction/supplemental_files/stock_prices.csv")
average = pd.DataFrame(prices.groupby("SecuritiesCode").Target.mean())
prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)

In [ ]:
X, y, codes = data_sets(prices)
print(X)

In [ ]:
# model = RandomForestRegressor(max_leaf_nodes=1000 ,random_state=1)
model = DecisionTreeRegressor(max_depth=30)
#model=LGBMRegressor(num_leaves=800, learning_rate=0.05, n_estimators=100)
model.fit(X, y)
model.score(X, y)

In [ ]:
%%time
import jpx_tokyo_market_prediction
env = jpx_tokyo_market_prediction.make_env()
iter_test = env.iter_test()

predictions = run_model

for prices, options, financials, trades, secondary_prices, sample_prediction in iter_test:
    
    prices["Avg"] = prices["SecuritiesCode"].apply(get_avg)
    df = predict(prices)
    prices.loc[:,"pred"] = predictions(df)
    prices = add_rank(prices)
    rank = prices.set_index('SecuritiesCode')['Rank'].to_dict()
    sample_prediction['Rank'] = sample_prediction['SecuritiesCode'].map(rank)
    env.predict(sample_prediction)